In [227]:
import os
from pathlib import Path

import pandas as pd


## Load + Do Data Exploration

In [228]:
# traverse to root of repo dynamically
cwd = Path().cwd()
if "_FLAG_CWD_SET" not in globals():
  while cwd.name != "src":
    print("looking for repo root:", cwd.parent)
    cwd = cwd.parent
  os.chdir(cwd.parent)
print("In repo root:", Path().cwd())
_FLAG_CWD_SET = True

In repo root: /home/will/cs5100-project


In [229]:
# this assumes that you have already run the paprika parser code to code
# (see 'make build' for more details)
INPUT_PATH: Path = Path("resources/paprika/.export.paprikarecipes.parsed.json")
OUTPUT_PATH: Path = INPUT_PATH.parent / ".export.paprikarecipes.cleaned.json"
INPUT_PATH.absolute()

PosixPath('/home/will/cs5100-project/resources/paprika/.export.paprikarecipes.parsed.json')

In [230]:
df = pd.read_json(INPUT_PATH)
df = df.replace(r"^\s*$", pd.NA, regex=True)

In [231]:
print(f"{df.shape=}\n\n{df.columns=}")

df.shape=(424, 24)

df.columns=Index(['uid', 'created', 'hash', 'name', 'description', 'ingredients',
       'directions', 'notes', 'nutritional_info', 'prep_time', 'cook_time',
       'total_time', 'difficulty', 'servings', 'rating', 'source',
       'source_url', 'photo', 'photo_large', 'photo_hash', 'image_url',
       'photo_data', 'photos', 'categories'],
      dtype='object')


In [232]:
# from above we can see there may be some columns we don't need since this is purely a
# text based GUI/TUI the user will be talking with
df = df.drop(
  columns=[
    "photo_hash",
    "photos",
    "photo",
    "image_url",
    "photo_large",
    "photo_data",
    "hash",
    "uid",
  ]
)

print(f"{df.columns=}")

df.columns=Index(['created', 'name', 'description', 'ingredients', 'directions', 'notes',
       'nutritional_info', 'prep_time', 'cook_time', 'total_time',
       'difficulty', 'servings', 'rating', 'source', 'source_url',
       'categories'],
      dtype='object')


### Summary Stats

In [233]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424 entries, 0 to 423
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   created           424 non-null    object
 1   name              424 non-null    object
 2   description       45 non-null     object
 3   ingredients       398 non-null    object
 4   directions        399 non-null    object
 5   notes             71 non-null     object
 6   nutritional_info  184 non-null    object
 7   prep_time         237 non-null    object
 8   cook_time         237 non-null    object
 9   total_time        216 non-null    object
 10  difficulty        219 non-null    object
 11  servings          350 non-null    object
 12  rating            424 non-null    int64 
 13  source            397 non-null    object
 14  source_url        290 non-null    object
 15  categories        424 non-null    object
dtypes: int64(1), object(15)
memory usage: 53.1+ KB


In [234]:
df.describe(include="all")  # for object columns

,created,name,description,ingredients,directions,notes,nutritional_info,prep_time,cook_time,total_time,difficulty,servings,rating,source,source_url,categories
count,424,424,45,398,399,71,184,237,237,216,219,350,424.000000,397,290,424
unique,424,424,45,398,397,71,184,44,82,54,3,122,NaN,100,288,151
top,2022-12-03 17:55:09,"_Assorted Drinks Wines, Good Ones",Opal Apples - Idylwilde Acton April 2023\nAtaulfo Mangos - Idylwilde Acton April 2023,[See additional photos],[See additional photos],"*Skip water bc too much liquid --> flat cookies\n**originally 1tsp, but deemed coffee taste as overwhelming so cut down",Calories 260\nFat 15g\nCarbs 28g\nFiber 1g\nSugar 16g\nProtein 3g,5 min,30 min,40 min,Easy,5,NaN,cooking.nytimes.com,https://www.blueapron.com/recipes/italian-style-pork-meatloaf-with-lemon-caper-roasted-vegetables-08bc7c5c-8fd9-43f5-8ee5-f14ea4d9280d,[1'Main Dish]
freq,1,1,1,1,3,1,1,37,20,21,197,46,NaN,120,2,66
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.162736,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.313375,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.000000,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.000000,NaN,NaN,NaN


In [235]:
# This code-block is from MISM3415 (data-mining) lecture
MAX_VALUES_TO_SHOW = 5

# for each column
for col in df.columns:
  print(col)

  # get a list of unique values, handling unhashable types like lists
  try:
    unique = df[col].unique()
  except TypeError:
    unique = pd.Series(
      df[col].apply(lambda x: str(x) if isinstance(x, list) else x)
    ).unique()

  # if number of unique values is less than 30, print the values. Otherwise print the
  # number of unique values
  # if len(unique) < MAX_VALUES_TO_SHOW:
  #   print(unique, "\n====================================\n\n")
  # else:

  print(unique[:MAX_VALUES_TO_SHOW])

  if len(unique) > MAX_VALUES_TO_SHOW:
    print(
      str(len(unique) - MAX_VALUES_TO_SHOW) + "more unique values",
      "\n====================================\n\n",
    )


created
['2022-12-03 17:55:09' '2023-04-08 17:08:59' '2023-10-06 18:29:49'
 '2023-06-24 09:40:30' '2023-06-21 10:18:47']
419more unique values 


name
['_Assorted Drinks Wines, Good Ones' '_Assorted Fruit Mango, Apples'
 '_Assorted Good Stuff' '_Assorted Sauces, Asian Sauces'
 '_Assorted Spices, Cuban Goya Chicken Bouillon']
419more unique values 


description
[<NA>
 'Opal Apples - Idylwilde Acton April 2023\nAtaulfo Mangos - Idylwilde Acton April 2023'
 'Yams Idylwilde April 2023'
 'Alton Brown or La Terra Fina (Market Basket)'
 'Make with [recipe:Pie Flaky Pie Crust], but this adds an extra 4.5 hrs to time! So if making for thanksgiving make first!']
41more unique values 


ingredients
['[See additional photos]' <NA>
 '1 cup bread flour\n¾ cup all-purpose flour\n2 teaspoons kosher salt, or 1½ teaspoons table salt\n1 teaspoon baking soda\n1 cup unsalted butter (2 sticks)\n2 tablespoons water, room temperature*\n1 cup dark brown sugar\n½ cup white sugar\n2 teaspoons vanilla extract\n1

## Cleansing

In [236]:
# Add new column for "been tried"
df["been_tried"] = df["rating"].apply(
  lambda x: False if x is None or x == "0" else True
)

In [237]:
# created currently str '2022-12-03 17:55:09' -> DateTime
from datetime import datetime

df["created"] = df["created"].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))

In [238]:
pd.set_option("display.max_colwidth", 1000)
df["name"][df["name"].apply(lambda x: not x.isascii() or not x[0].isalpha())]

0                                   _Assorted Drinks Wines, Good Ones
1                                       _Assorted Fruit Mango, Apples
2                                                _Assorted Good Stuff
3                                      _Assorted Sauces, Asian Sauces
4                       _Assorted Spices, Cuban Goya Chicken Bouillon
5                                                _Assorted Vegetables
6                    !!!*THE* Chocolate Chip Cookie Recipe (Lauren's)
7                         🦃 Dip Spinach Artichoke Dip (La Terra Fina)
8                                    🦃 Pie Libby's Famous Pumpkin Pie
9               🦃Bacon-Wrapped Brussels Sprouts with Creamy Lemon Dip
10                            🦃Dip Baked Brie Cranberry in Bread Bowl
11                               🦃Drink Apple Cider and Bourbon Punch
12                                    🦃Drink Apple Cider Apple Brandy
13                                     🦃Drink Cranberry Gin and Tonic
14                  

In [239]:
# drop rows of df[name] where starts with "_" since these are
# categories or labels in the given dataset
mask = df["name"].str.startswith("_")
print(f"dropping {mask.sum()} placeholder recipes")
df = df.loc[~mask].reset_index(drop=True)


dropping 6 placeholder recipes


In [240]:
# get rid of "fake" recipes
df["clean_name"] = df["name"]

non_ascii_subsitutions = {
  "’": "'",
  "‘": "'",
  "🦃": "(Thanksgiving)",
  "😋": " ",
  "🍪": " ",
  ",": ",",
  "!!!*THE*": "The",
}

# make character subsitutions for non_ascii_substitutions in df["clean_name"]
for src, dst in non_ascii_subsitutions.items():
  df["clean_name"] = df["clean_name"].str.replace(src, dst, regex=False)


df["clean_name"][df["clean_name"].apply(lambda x: not x.isascii())]

26                                       Apple Crumble à la Air Fryer
178                                        Cuban-ish Pollo en Fricasé
323            Pork Glazed Grape and Jalapeño Roasted Pork Tenderloin
330                             Pudding Vanilla Crème Brûlée Lauren's
363    Seafood Provençal Salmon with Fennel, Rosemary and Orange Zest
Name: clean_name, dtype: object

In [241]:
# categories is string of "["1\'Drinks + Cocktails", \'_Try these\']",
starting_tags = set()
ending_tags = set()


def parse_categories(s: list[str]) -> list[str]:
  cleaned = []
  for tag in s:
    starting_tags.add(tag)
    tag = tag.strip()

    # remove numeric prefix like 1' or 2'
    tag = tag.replace("’", "'")
    if "'" in tag and tag[0].isnumeric():
      tag = tag.split("'", 1)[1].strip()
    # remove leading underscore used for internal tags
    if tag.startswith("_"):
      tag = tag[1:]

    if len(tag) == 0:
      continue

    ending_tags.add(tag)
    cleaned.append(tag)
  return cleaned


# create a new column with parsed category lists
df["categories_parsed"] = df["categories"].apply(parse_categories)
print(f"{starting_tags=},\n {ending_tags=}")

starting_tags={"1'Dessert", '2’Cake', "3'Cuban", "1'Breakfast", "2'Salad", "1'Appetizer + Snacks", "1'Drinks + Cocktails", "3'Air Fryer", "_Lauren's", "New Year's Eve + Day", "2'Vegetables + Fruits", "3'Holidays", 'Noche Buena + Christmas', 'Thanksgiving', "2'Pasta", 'Easter', "1'Side Dish", "_William's", "1'Main Dish", '_Try these', "2'Rice", "_Perli's", "2'Soup Stews and Beans", "2'Sandwiches", "St Patrick's", "2'Bread"},
 ending_tags={'Sandwiches', 'Cake', 'Pasta', "Perli's", 'Breakfast', 'Cuban', 'Appetizer + Snacks', 'Dessert', "New Year's Eve + Day", 'Salad', "Lauren's", 'Bread', 'Soup Stews and Beans', 'Holidays', 'Noche Buena + Christmas', 'Thanksgiving', 'Air Fryer', 'Rice', 'Main Dish', 'Easter', 'Vegetables + Fruits', "William's", 'Try these', "St Patrick's", 'Side Dish', 'Drinks + Cocktails'}


In [242]:
time_columns = ["prep_time", "cook_time", "total_time"]

for time_column in time_columns:
  print(f"\ncreating duration in minutes column for '{time_column}'")

  # try to convert disparate time formats
  temp = df[time_column].str.replace("mins", "minutes")
  temp = temp.str.replace("hrs", "hours")

  # remove bad formats
  temp = temp.mask(
    temp.str.contains(r"\b(?:or|chilling)\b", case=False, na=False), pd.NA
  )
  temp = temp.mask(temp.str.contains(r"[-–—\\\/:]", na=False), pd.NA)

  # show the amount of rows which could not be converted
  print(
    f"ignoring {temp.isna().sum() - df[time_column].isna().sum()}"
    " rows due to poor formatting"
  )

  # do basic time conversion
  temp = (pd.to_timedelta(temp).dt.total_seconds() // 60).astype(pd.Int64Dtype())
  df[f"{time_column}_min"] = temp



creating duration in minutes column for 'prep_time'
ignoring 1 rows due to poor formatting

creating duration in minutes column for 'cook_time'
ignoring 7 rows due to poor formatting

creating duration in minutes column for 'total_time'
ignoring 0 rows due to poor formatting


In [243]:
df = df.dropna(
  subset=["notes", "directions", "description", "ingredients", "clean_name"],
  how="all",
)

In [244]:
df = df.convert_dtypes()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   created            418 non-null    datetime64[ns]
 1   name               418 non-null    string        
 2   description        43 non-null     string        
 3   ingredients        397 non-null    string        
 4   directions         398 non-null    string        
 5   notes              71 non-null     string        
 6   nutritional_info   184 non-null    string        
 7   prep_time          237 non-null    string        
 8   cook_time          237 non-null    string        
 9   total_time         216 non-null    string        
 10  difficulty         219 non-null    string        
 11  servings           350 non-null    string        
 12  rating             418 non-null    Int64         
 13  source             396 non-null    string        
 14  source_url

In [245]:
df.describe(include="all")

,created,name,description,ingredients,directions,notes,nutritional_info,prep_time,cook_time,total_time,...,rating,source,source_url,categories,been_tried,clean_name,categories_parsed,prep_time_min,cook_time_min,total_time_min
count,418,418,43,397,398,71,184,237,237,216,...,418.0,396,290,418,418,418,418,236.0,230.0,216.0
unique,NaN,418,43,397,397,71,184,44,82,54,...,<NA>,100,288,146,1,418,146,<NA>,<NA>,<NA>
top,NaN,!!!*THE* Chocolate Chip Cookie Recipe (Lauren's),Alton Brown or La Terra Fina (Market Basket),"1 cup bread flour\n¾ cup all-purpose flour\n2 teaspoons kosher salt, or 1½ teaspoons table salt\n1 teaspoon baking soda\n1 cup unsalted butter (2 sticks)\n2 tablespoons water, room temperature*\n1 cup dark brown sugar\n½ cup white sugar\n2 teaspoons vanilla extract\n1/2 teaspoon espresso powder**\n1 large egg\n1 large egg yolk\n½ cup semi-sweet chocolate chips or chunks\n5 oz dark chocolate, chopped (Ghirardelli 60% Cacao)",[See additional photos],"*Skip water bc too much liquid --> flat cookies\n**originally 1tsp, but deemed coffee taste as overwhelming so cut down",Calories 260\nFat 15g\nCarbs 28g\nFiber 1g\nSugar 16g\nProtein 3g,5 min,30 min,40 min,...,<NA>,cooking.nytimes.com,https://www.blueapron.com/recipes/italian-style-pork-meatloaf-with-lemon-caper-roasted-vegetables-08bc7c5c-8fd9-43f5-8ee5-f14ea4d9280d,[1'Main Dish],True,The Chocolate Chip Cookie Recipe (Lauren's),[Main Dish],<NA>,<NA>,<NA>
freq,NaN,1,1,1,2,1,1,37,20,21,...,<NA>,120,2,66,418,1,66,<NA>,<NA>,<NA>
mean,2023-07-21 14:18:30.208134144,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.19378,NaN,NaN,NaN,NaN,NaN,NaN,21.09322,36.469565,67.37037
min,2022-11-26 09:25:46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
25%,2022-11-27 10:50:42.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,12.0,25.0
50%,2023-03-29 05:14:14.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,10.0,25.0,40.0
75%,2024-03-24 19:24:44.750000128,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,15.0,40.0,75.0
max,2025-05-30 19:35:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,1440.0,525.0,1290.0


In [246]:
df.to_json(OUTPUT_PATH, index=False, orient="records", indent=2)